In [ ]:
cd drive/MyDrive/Colab\ Notebooks/GSOC\ 2021/ML4Sci

/content/drive/MyDrive/Colab Notebooks/GSOC 2021/ML4Sci


In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import h5py
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer, StandardScaler
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from sklearn.metrics import accuracy_score

from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

In [ ]:

filename='Electron.hdf5'
data1 = h5py.File(filename, 'r')
Y1=data1['y']
X1=data1['X']

In [ ]:
filename='Photon.hdf5'
data0 = h5py.File(filename, 'r')
Y0=data0['y']
X0=data0['X']

In [ ]:
X_final=np.concatenate((X0[:],X1[:]),axis=0)
Y_final=np.concatenate((Y0[:],Y1[:]),axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid, Y_train, Y_valid = train_test_split(X_final[:,:,:,0].reshape((X_final.shape[0],1,X_final.shape[1],X_final.shape[2])),Y_final,test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)

(398400, 1, 32, 32) (398400,)
(99600, 1, 32, 32) (99600,)


In [ ]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, inputs, labels):
        'Initialization'
        self.labels = labels
        self.inputs = inputs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs)

  def __getitem__(self, index):
        'Generates one sample of data'

        # Get data and get label
        X = self.inputs[index]
        y = self.labels[index]

        return X, y

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset={'input':X_train,'output':Y_train},
                                          batch_size=100,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset={'input':X_valid},
                                          batch_size=100,
                                          shuffle=False)

In [ ]:

window_height=32
window_width=32

In [ ]:

import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        #self.relu1 = nn.Relu()
        
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        #self.relu2 = nn.Relu()
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
       # self.pool = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(4, 4)
        #self.fc1 = nn.Flatten()
        #self.relu3 = nn.Relu()
        #self.fc1 = nn.Linear(256, 128)
        #self.relu4 = nn.Relu()
        #self.fc3 = nn.Linear(1024, 256)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        #self.fc2 = nn.Linear(128, 1)
        #self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        #x = F.relu(self.conv4(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
       # x = F.relu(self.conv6(x))
        x = self.pool1(F.relu(self.conv6(x)))
        #x = self.fc1()
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
       # x = F.leaky_relu(self.fc3(x))
        x = F.sigmoid(self.fc2(x))
        return x


net = Net()
net.cuda()
summary(net, (1, 32, 32), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]              80
            Conv2d-2            [-1, 8, 32, 32]             584
         MaxPool2d-3            [-1, 8, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]           2,336
            Conv2d-5           [-1, 32, 16, 16]           9,248
         MaxPool2d-6             [-1, 32, 8, 8]               0
            Conv2d-7             [-1, 32, 8, 8]           9,248
            Conv2d-8             [-1, 32, 8, 8]           9,248
         MaxPool2d-9             [-1, 32, 2, 2]               0
           Linear-10                   [-1, 64]           8,256
           Linear-11                    [-1, 1]              65
Total params: 39,065
Trainable params: 39,065
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
# ## Define CNN model
model = Sequential()
model.add(Conv2D(8, activation='relu', kernel_size=3, padding='same', kernel_initializer='GlorotNormal', input_shape=(img_rows, img_cols, nb_channels)))
model.add(Conv2D(8, activation='relu', kernel_size=3, padding='same', kernel_initializer='GlorotNormal'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, activation='relu', kernel_size=3, padding='same', kernel_initializer='GlorotNormal'))
model.add(Conv2D(32, activation='relu', kernel_size=3, padding='same', kernel_initializer='GlorotNormal'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, activation='relu', kernel_size=5, padding='same', kernel_initializer='GlorotNormal'))
model.add(Conv2D(32, activation='relu', kernel_size=5, padding='same', kernel_initializer='GlorotNormal'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='GlorotNormal'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='GlorotNormal'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid', kernel_initializer='GlorotNormal'))
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr_init), metrics=['accuracy'])
model.summary()

In [ ]:
use_cuda = True
if use_cuda and torch.cuda.is_available():
    net.cuda()

In [ ]:
batch_size=1024
import torch
from torch.utils import data
#import cudnn
#from my_classes import Dataset


# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True

# Parameters
params = {'batch_size': batch_size,
          'shuffle': True,
          }
max_epochs = 100

# Generators
training_set = Dataset(X_train, Y_train)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(X_valid, Y_valid)
validation_generator = data.DataLoader(validation_set, **params)

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
num_epochs=40
correct=0
total=0
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(training_generator):   # Load a batch of images with its (index, data, class)
        #images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        #labels = Variable(labels)
        
        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(inputs)
                                      # Forward pass: compute the output class given a image
        loss = criterion(outputs[:,0], labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        outputs[outputs>0.5]=1
        outputs[outputs<0.5]=0
        total += labels.shape[0]
                                        # Increment the total count
        correct += (outputs[:,0] == labels).sum()
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.4f'
                 %(epoch+1, num_epochs, i+1, len(X_train)//batch_size, loss.item(), ((100 * correct.item() / total))))
    #outputs = net(X_valid0.cuda())                             # Forward pass: compute the output class given a image
    #loss = criterion(outputs, Y_valid.cuda())

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/40], Step [100/389], Loss: 0.6781, Accuracy: 51.5498
Epoch [1/40], Step [200/389], Loss: 0.6609, Accuracy: 55.1655
Epoch [1/40], Step [300/389], Loss: 0.6469, Accuracy: 56.7878
Epoch [2/40], Step [100/389], Loss: 0.6155, Accuracy: 59.4880
Epoch [2/40], Step [200/389], Loss: 0.5968, Accuracy: 60.9577
Epoch [2/40], Step [300/389], Loss: 0.5986, Accuracy: 62.1689
Epoch [3/40], Step [100/389], Loss: 0.5732, Accuracy: 63.7423
Epoch [3/40], Step [200/389], Loss: 0.5738, Accuracy: 64.4247
Epoch [3/40], Step [300/389], Loss: 0.5844, Accuracy: 64.9768
Epoch [4/40], Step [100/389], Loss: 0.5752, Accuracy: 65.7982
Epoch [4/40], Step [200/389], Loss: 0.5832, Accuracy: 66.1797
Epoch [4/40], Step [300/389], Loss: 0.5966, Accuracy: 66.4859
Epoch [5/40], Step [100/389], Loss: 0.5778, Accuracy: 66.9699
Epoch [5/40], Step [200/389], Loss: 0.5633, Accuracy: 67.1867
Epoch [5/40], Step [300/389], Loss: 0.5737, Accuracy: 67.4059
Epoch [6/40], Step [100/389], Loss: 0.5589, Accuracy: 67.7669
Epoch [6

In [ ]:
torch.save(net.state_dict(), 'pytorch2.pt')

In [ ]:
net = Net()
net.load_state_dict(torch.load('pytorch2.pt'))
if use_cuda and torch.cuda.is_available():
    net.cuda()
net.eval()

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
pred=torch.Tensor.cpu(net(inputs[0:10000])).detach().numpy()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:

true_vals=torch.Tensor.cpu(labels[:10000]).detach().numpy()

In [ ]:
from sklearn.metrics import roc_curve
y_pred = pred.ravel()
fpr, tpr, thresholds = roc_curve(true_vals, y_pred)

In [ ]:

from sklearn.metrics import auc
auc_keras = auc(fpr, tpr)

In [ ]:
score = nn.eval(X_valid, y_valid, verbose=1)
print('\nValidation loss / accuracy: %0.4f / %0.4f'%(score[0], score[1]))
y_pred = nn.predict(X_valid)
fpr, tpr, _ = roc_curve(y_valid, y_pred)
roc_auc = auc(fpr, tpr)
print('Validation ROC AUC:', roc_auc)

# Evaluate on test set
score = model.evaluate(X_test, y_test, verbose=1)
print('\nTest loss / accuracy: %0.4f / %0.4f'%(score[0], score[1]))
y_pred = model.predict(X_test)
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
print('Test ROC AUC:', roc_auc)